## Preprocessing the Data

In [20]:
import warnings
warnings.filterwarnings('ignore')

# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [21]:
w = application_df["STATUS"]
w.value_counts()

1    34294
0        5
Name: STATUS, dtype: int64

#### EIN, NAME

In [22]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME' 

application_df = application_df.drop(["EIN", "NAME"], axis=1)

In [23]:
# Determine the number of unique values in each column

application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

#### APPLICATION_TYPE

In [24]:
type_count = application_df.APPLICATION_TYPE.value_counts()

# Determine which values to replace if counts are less than 50
replace_application = list(type_count[type_count < 50].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
Other       54
Name: APPLICATION_TYPE, dtype: int64

#### CLASSIFICATION

In [25]:
class_count = application_df.CLASSIFICATION.value_counts()

# Determine which values to replace if counts are less than 100
replace_class = list(class_count[class_count < 100].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [26]:
# Generate our categorical variable lists

app_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_df[app_cat].nunique()

APPLICATION_TYPE          11
AFFILIATION                6
CLASSIFICATION            12
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

### OneHotEncoder and StandardScaler

In [27]:
# Generate categorical variable list again

app_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_df[app_cat].nunique()

APPLICATION_TYPE          11
AFFILIATION                6
CLASSIFICATION            12
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [28]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[app_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(app_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [29]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(app_cat,1)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [30]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [31]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Logistic Regression

In [32]:
# Define the logistic regression model
log_classifier = LogisticRegression(solver="lbfgs",max_iter=1000)

# Train the model
log_classifier.fit(X_train,y_train)

LogisticRegression(max_iter=1000)

In [33]:
# Evaluate the model - Logistic Regression
y_pred = log_classifier.predict(X_test)
print(f" Logistic regression model accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Logistic regression model accuracy: 0.467


## Random Forest 

In [40]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=100)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [41]:
# Evaluate the model - Random Forest
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Random forest predictive accuracy: 0.709


Using n_estimators=100 or n_estimators=1000 the accuracy is the same: 0.709

## Deep Learning Model

First hidden layer: units=200
Second hidden layer: units=100

In [42]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()


# First hidden layer
nn.add(tf.keras.layers.Dense(units=200, input_dim=51, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=100, activation="relu"))



# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 200)               10400     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               20100     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 30,601
Trainable params: 30,601
Non-trainable params: 0
_________________________________________________________________


2021-11-07 12:53:30.632822: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [43]:
# Compile the model

nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [44]:
# Train the model 

fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

2021-11-07 12:53:35.422362: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5675 - accuracy: 0.7245
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5529 - accuracy: 0.7317
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5498 - accuracy: 0.7313
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5485 - accuracy: 0.7339
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5470 - accuracy: 0.7340
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5457 - accuracy: 0.7348
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5457 - accuracy: 0.7346
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5443 - accuracy: 0.7350
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5437 - accuracy: 0.7345
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5439 - accura

804/804 [==============================] - 1s 2ms/step - loss: 0.5312 - accuracy: 0.7421
Epoch 82/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5318 - accuracy: 0.7425
Epoch 83/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5316 - accuracy: 0.7418
Epoch 84/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5314 - accuracy: 0.7422
Epoch 85/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5313 - accuracy: 0.7425
Epoch 86/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5312 - accuracy: 0.7421
Epoch 87/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5311 - accuracy: 0.7425
Epoch 88/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5311 - accuracy: 0.7428
Epoch 89/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5311 - accuracy: 0.7424
Epoch 90/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5316 - accuracy: 

In [45]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5635 - accuracy: 0.7271
Loss: 0.5635430812835693, Accuracy: 0.7271137237548828


## Deep Learning Model

3 hidden layers

In [47]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()


# First hidden layer
nn.add(tf.keras.layers.Dense(units=200, input_dim=51, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=100, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=50, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 200)               10400     
_________________________________________________________________
dense_4 (Dense)              (None, 100)               20100     
_________________________________________________________________
dense_5 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 51        
Total params: 35,601
Trainable params: 35,601
Non-trainable params: 0
_________________________________________________________________


In [48]:
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5651 - accuracy: 0.7236
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5540 - accuracy: 0.7304
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5510 - accuracy: 0.7336
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5488 - accuracy: 0.7343
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5468 - accuracy: 0.7338
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5466 - accuracy: 0.7349
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5455 - accuracy: 0.7354
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5441 - accuracy: 0.7360
Epoch 9/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5442 - accuracy: 0.7357
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5428 - accura

804/804 [==============================] - 2s 2ms/step - loss: 0.5300 - accuracy: 0.7427
Epoch 81/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5303 - accuracy: 0.7421
Epoch 82/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5306 - accuracy: 0.7422
Epoch 83/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5300 - accuracy: 0.7424
Epoch 84/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5298 - accuracy: 0.7425
Epoch 85/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5295 - accuracy: 0.7428
Epoch 86/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5299 - accuracy: 0.7425
Epoch 87/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5314 - accuracy: 0.7417
Epoch 88/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5293 - accuracy: 0.7427: 0s - loss: 0.5296 - accuracy: 0.74
Epoch 89/100
804/804 [==============================] - 1s 

In [49]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5671 - accuracy: 0.7252
Loss: 0.567095160484314, Accuracy: 0.7252478003501892


## Deep Learning Model

Using Tanh in the second hidden layer 

In [50]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()


# First hidden layer
nn.add(tf.keras.layers.Dense(units=200, input_dim=51, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=100, activation="tanh"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 200)               10400     
_________________________________________________________________
dense_8 (Dense)              (None, 100)               20100     
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 101       
Total params: 30,601
Trainable params: 30,601
Non-trainable params: 0
_________________________________________________________________


In [51]:
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5694 - accuracy: 0.7209
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5561 - accuracy: 0.7301
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5524 - accuracy: 0.7295
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5506 - accuracy: 0.7317
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5489 - accuracy: 0.7333
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5482 - accuracy: 0.7316
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5476 - accuracy: 0.7324
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5461 - accuracy: 0.7322
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5458 - accuracy: 0.7346
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5448 - accura

804/804 [==============================] - 1s 2ms/step - loss: 0.5344 - accuracy: 0.7400
Epoch 81/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5341 - accuracy: 0.7403
Epoch 82/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5346 - accuracy: 0.7403: 0s - loss: 0.5
Epoch 83/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5343 - accuracy: 0.7413
Epoch 84/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5340 - accuracy: 0.7397
Epoch 85/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5340 - accuracy: 0.7414: 0s - loss: 0.5349 - accuracy: 0.
Epoch 86/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5336 - accuracy: 0.7411
Epoch 87/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5334 - accuracy: 0.7399
Epoch 88/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5343 - accuracy: 0.7403
Epoch 89/100
804/804 [=======================

In [52]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5531 - accuracy: 0.7230
Loss: 0.5531148910522461, Accuracy: 0.7230320572853088


## Deep Learning Model

First hidden layer: units=300
Second hidden layer: units=150
Epochs = 200

In [53]:
nn = tf.keras.models.Sequential()


# First hidden layer
nn.add(tf.keras.layers.Dense(units=300, input_dim=51, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=150, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 300)               15600     
_________________________________________________________________
dense_11 (Dense)             (None, 150)               45150     
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 151       
Total params: 60,901
Trainable params: 60,901
Non-trainable params: 0
_________________________________________________________________


In [54]:
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

fit_model = nn.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5675 - accuracy: 0.7235
Epoch 2/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5537 - accuracy: 0.7308
Epoch 3/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5505 - accuracy: 0.7317
Epoch 4/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5489 - accuracy: 0.7313
Epoch 5/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5467 - accuracy: 0.7346
Epoch 6/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5466 - accuracy: 0.7348
Epoch 7/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5455 - accuracy: 0.7356: 0s - loss: 0.5
Epoch 8/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5448 - accuracy: 0.7352
Epoch 9/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5443 - accuracy: 0.7356
Epoch 10/200
804/804 [==============================] - 2s 2ms/step - loss:

804/804 [==============================] - 2s 2ms/step - loss: 0.5326 - accuracy: 0.7429
Epoch 80/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5317 - accuracy: 0.7422
Epoch 81/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5313 - accuracy: 0.7427
Epoch 82/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5312 - accuracy: 0.7420
Epoch 83/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5307 - accuracy: 0.7420
Epoch 84/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5316 - accuracy: 0.7419
Epoch 85/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5315 - accuracy: 0.7421
Epoch 86/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5309 - accuracy: 0.7425
Epoch 87/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5306 - accuracy: 0.7422
Epoch 88/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5306 - accuracy: 

804/804 [==============================] - 2s 2ms/step - loss: 0.5322 - accuracy: 0.7432
Epoch 158/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5292 - accuracy: 0.7434
Epoch 159/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5291 - accuracy: 0.7434
Epoch 160/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5301 - accuracy: 0.7425
Epoch 161/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5298 - accuracy: 0.7429
Epoch 162/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5291 - accuracy: 0.7429
Epoch 163/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5286 - accuracy: 0.7435
Epoch 164/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5286 - accuracy: 0.7429
Epoch 165/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5292 - accuracy: 0.7429
Epoch 166/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5296 - a

In [55]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5840 - accuracy: 0.7261
Loss: 0.5839621424674988, Accuracy: 0.726064145565033
